In [3]:
#Import our libraries
import time
import requests
import json
import xml.etree.ElementTree as ET  # for parsing XML
import xmltodict
import pandas as pd
import csv
import base64 
import pyodbc
import sqlalchemy as sal
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR

#Import our helper functions
import helper_functions

# Step 1: View the list of reports available

In [4]:
report_lst = helper_functions.get_available_reports()

In [5]:
#Turn it into a list to see all of them
list(report_lst['ReportName'])

['_Current Employee Deductions and Benefits',
 '2013-03-05T21:47:40.188Z',
 '2020-10-26T18:25:32.678Z',
 '2021 Allocation Report (1)',
 '401k Allocation Report V1',
 '401k Allocation Report V2',
 '401k Allocation Report V2_UKG',
 '401k Allocation Report V3_UKG',
 '401k Allocation Report V3_UKG v2',
 '401k Allocation Report V3_UKG v4',
 '5 Level Supervisors',
 'Active EE_DD List',
 'Allocated Hours FTE by Date',
 'Allocated Hours FTE by Month',
 'Annual Earnings',
 'Audit Report with Time vs Pay Matrix Tables',
 'Comp Project V3',
 'Configuration Validation',
 'Counts by Location',
 'Current Employee Deductions and Benefits',
 "David's Report",
 "David's Report V2",
 'Deduction list',
 'Deduction Pay History By Deduction For EmployeesUKG',
 'Deductions in Arrears V2',
 'Deferred Comp Wages and Hours_UKG',
 'Dependents by Employee and Plan test',
 'Document Category Report',
 'DSHS_New Hires',
 'Earning Codes Table',
 'Earnings Pay History By Earning Code For Employees_ULTI',
 'Earnings_

In [6]:
#Get the precheck deduction report pathname
report_lst[report_lst['ReportName'].str.contains('employee', regex=True, case=False)]

,ReportName,ReportPath
0,_Current Employee Deductions and Benefits,/content/folder[@name='zzzCompany Folders']/fo...
19,Current Employee Deductions and Benefits,/content/folder[@name='zzzCompany Folders']/fo...
23,Deduction Pay History By Deduction For Employe...,/content/folder[@name='zzzCompany Folders']/fo...
26,Dependents by Employee and Plan test,/content/folder[@name='zzzCompany Folders']/fo...
30,Earnings Pay History By Earning Code For Emplo...,/content/folder[@name='zzzCompany Folders']/fo...
35,Employee job history report,/content/folder[@name='zzzCompany Folders']/fo...
36,Employee List,/content/folder[@name='zzzCompany Folders']/fo...
37,Employee Location Changes v2,/content/folder[@name='zzzCompany Folders']/fo...
38,Employee PTO History - Balances - Updated 2,/content/folder[@name='zzzCompany Folders']/fo...
39,Employee Waived Deductions,/content/folder[@name='zzzCompany Folders']/fo...


In [7]:
report_path = report_lst[report_lst['ReportName'] == 'Employee List']['ReportPath'].loc[36]
report_path

"/content/folder[@name='zzzCompany Folders']/folder[@name='Proliance Surgeons, Inc., P.S.']/folder[@name='UltiPro']/folder[@name='Export to SQL']/report[@name='Employee List']"

## Step 2: Retrieve the report

In [8]:
df = helper_functions.retrieve_report(report_path) 
df 

c:\UKG\Reports as a service\helper_functions.py:367: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].astype(str).str.replace('{', '')
c:\UKG\Reports as a service\helper_functions.py:368: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].astype(str).str.replace('}', '')
c:\UKG\Reports as a service\helper_functions.py:369: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].astype(str).str.replace('[', '')
c:\UKG\Reports as a service\helper_functions.py:370: FutureWarning: The defa

,pay_group,ssn_(unformatted),employee_number,"employee_name_(last_suffix,_first_mi)",address_line_1_+_address_line_2,address_line_1,address_line_2,city,state,zip_code,...,annual,first_name,last_name,middle_name,supervisor_name_(first_mi_last_suffix),supervisor_employee_number,supervisor_email_address,employee_email,alternate_email,home_phone
0,Hourly BiWeekly,534118377,001003651,Aarestad_ Heidi H.,7320 Highland Dr,7320 Highland Dr,None,Everett,WA,98203,...,61672,Heidi,Aarestad,H,Shelly K. Peltier,000250048,S.Peltier@proliancesurgeons.com,heidiaarestad@gmail.com,'@xs:nil': 'true',2064594160
1,Hourly BiWeekly,534153592,001007204,Aasal_ Ashley E.,22500 10th Ave S,22500 10th Ave S,'@xs:nil': 'true',Des Moines,WA,98198,...,56284.8,Ashley,Aasal,E,Jessica L. Weathers,000260032,J.Weathers@proliancesurgeons.com,A.Aasal@proliancesurgeons.com,ashleydamon24@gmail.com,2066603749
2,Hourly BiWeekly,607574894,001004881,Abad_ Queen Jacqueline D.,306 Kirkland Pl SE,306 Kirkland Pl SE,'@xs:nil': 'true',Renton,WA,98056,...,32240,Queen Jacqueline,Abad,D,'@xs:nil': 'true','@xs:nil': 'true','@xs:nil': 'true','@xs:nil': 'true','@xs:nil': 'true',2066194279
3,Hourly BiWeekly,531965588,001008549,Abafo_ Johanna A.,18216 132nd Pl SE,18216 132nd Pl SE,'@xs:nil': 'true',Renton,WA,98058,...,0,Johanna,Abafo,A,Erik M. Mortensen,001006786,E.Mortensen@proliancesurgeons.com,j.abafo@proliancesurgeons.com,amoraj1434@gmail.com,'@xs:nil': 'true'
4,Hourly BiWeekly,535045731,001001279,Abalahin_ Julie A.,10721 MT Tacoma Dr SW,10721 MT Tacoma Dr SW,'@xs:nil': 'true',Tacoma,WA,98498,...,0,Julie,Abalahin,A,'@xs:nil': 'true','@xs:nil': 'true','@xs:nil': 'true',j.abalahin@proliancesurgeons.com,'@xs:nil': 'true',2533801201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9369,Hourly BiWeekly,605583592,001004641,Zuniga_ Maritza,210 S 18th St,210 S 18th St,None,Mount Vernon,WA,98274,...,35074,Maritza,Zuniga,'@xs:nil': 'true','@xs:nil': 'true','@xs:nil': 'true','@xs:nil': 'true',M.Zuniga@proliancesurgeons.com,'@xs:nil': 'true',3608407536
9370,Hourly BiWeekly,179501094,001005696,Zweber_ Mary F.,8204 Beverly Ln,8204 Beverly Ln,'@xs:nil': 'true',Everett,WA,98203,...,43680,Mary,Zweber,F,'@xs:nil': 'true','@xs:nil': 'true','@xs:nil': 'true',M.Zweber@proliancesurgeons.com,'@xs:nil': 'true',4252313557
9371,Hourly BiWeekly,534116910,001006118,Zwicky_ Amber D.,5209 NE 10th Pl,5209 NE 10th Pl,'@xs:nil': 'true',Renton,WA,98059,...,93600,Amber,Zwicky,D,Julia V. Stefoglo,001003126,J.Stefoglo@proliancesurgeons.com,A.Zwicky@proliancesurgeons.com,'@xs:nil': 'true',5053143559
9372,Hourly BiWeekly,531374469,001006766,Zwolski_ Jane E.,12910 SE 240th ST Unit B4,12910 SE 240th ST,Unit B4,Kent,WA,98031,...,54600,Jane,Zwolski,E,Sarah L. Peterson,001005009,SL.Peterson@proliancesurgeons.com,J.Hackett@proliancesurgeons.com,janezwolski@gmail.com,2534868763


# Step 3: Export the report to a CSV file

In [9]:
df.to_csv('ukg_bulk_insert.csv', index=False)


# Step 4: Insert the report into BiData2 as a table

In [10]:
helper_functions.create_sql_table_v2('ZZ_TEST_Employee_List')

# Step 5: Close the API connection

In [11]:
helper_functions.close_api_connection()

'<s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing"><s:Header><a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bidata/2/IBIDataService/LogOffResponse</a:Action></s:Header><s:Body><LogOffResponse xmlns="http://www.ultipro.com/dataservices/bidata/2"><LogOffResult xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><ServiceId i:nil="true"/><ClientAccessKey>PLAB4</ClientAccessKey><Token i:nil="true"/><Status>LoggedOff</Status><StatusMessage i:nil="true"/><InstanceKey i:nil="true"/></LogOffResult></LogOffResponse></s:Body></s:Envelope>'